

We create an instance of a skyjo_env environment, call reset() to initialize the game and list the available agents (players):


In [1]:
import numpy as np

from rlskyjo.environment import skyjo_env
from importlib import reload
reload(skyjo_env)

<module 'rlskyjo.environment.skyjo_env' from '/home/michi/skybo_rl/rlskyjo/environment/skyjo_env.py'>

In [2]:
skyjo_env_cfg = {"num_players": 3}
env_pettingzoo = skyjo_env.env(**skyjo_env_cfg)
env_pettingzoo.reset()

env_pettingzoo.agents, env_pettingzoo.agent_selection

def sample_place():
    return np.random.randint(0,23)
        
def sample_draw():
    return np.random.randint(24,25)


In [3]:
print(env_pettingzoo.agent_selection, "draw")
env_pettingzoo.step(sample_draw())
print(env_pettingzoo.agent_selection, "place")
env_pettingzoo.step(
    sample_place()
)

player_0 draw
player_0 place
[WARNING]: Illegal move made, game terminating with current player losing. 
obs['action_mask'] contains a mask of all legal moves that can be chosen.


In [4]:
def random_admissible_policy(observation, action_mask):
    """picks randomly an admissible action from the action mask"""
    return np.random.choice(
        np.arange(len(action_mask)),
        p= action_mask/np.sum(action_mask)
    )

assert 1 not in [random_admissible_policy(None,[1,0,1]) for _ in range(1000)] 

In [5]:
i_episode = 1  
while i_episode <= 1:
    i_episode  += 1
    env_pettingzoo.reset()
    for agent in env_pettingzoo.agent_iter(max_iter=600):        
        # get observation (state) for current agent:
        obs, reward, done, info = env_pettingzoo.last()
        
        print("training fct:", obs, reward, done, info)
        # perform q-learning with update_Q_value()
        # your code here
        
        env_pettingzoo.render()
        
        # store current state            
        if not done: 
            # choose action using epsilon_greedy_policy()
            # your code here    
            observation = obs["observations"]
            action_mask = obs["action_mask"]
            action = random_admissible_policy(observation, action_mask)
        
            print(f"sampled action {agent}: {action}")
            env_pettingzoo.step(action)
        else: 
            # agent is done
            env_pettingzoo.step(None)
            print('done', reward)
            break
    
    
else:
    print(env_pettingzoo._cumulative_rewards)

training fct: {'observations': array([-2, 10,  1,  1,  2,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  1,  0,
        0, 15, 15, 15, 11, 15, 15, 15, 15, 15, -1, 15, 15, 15, 15, 15, 15,
        9, 15, 15,  5, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       -2,  0, 15, 15], dtype=int8), 'action_mask': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1], dtype=int8)} 0 False {}
======= render board: ===== 
======= stats ============ 
next turn: draw by Player 1 
holding card player 1: empty 
discard pile top: 0 
======= Player 0 ========== 
[[u	 u	 u	 u]
 [u	 u	 u	 u]
 [11	 u	 -1	 u]]
======= Player 1 ========== 
[[u	 9	 5	 u]
 [u	 u	 u	 u]
 [u	 u	 u	 u]]
======= Player 2 ========== 
[[u	 u	 u	 0]
 [u	 u	 u	 u]
 [u	 u	 -2	 u]]

sampled action player_1: 24
training fct: {'observations': array([-2, 10,  1,  1,  2,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  1,  0,
        0,  3, 15, 15, 11, 15, 15, 15, 15, 15, -1, 15, 15, 15, 15, 15, 15,
        9

more envs test with rllib

In [6]:
from ray import init
init(num_cpus=os.cpu_count())

{'node_ip_address': '172.20.107.113',
 'raylet_ip_address': '172.20.107.113',
 'redis_address': '172.20.107.113:6379',
 'object_store_address': '/tmp/ray/session_2022-01-30_10-49-02_206427_299/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-01-30_10-49-02_206427_299/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-01-30_10-49-02_206427_299',
 'metrics_export_port': 48711,
 'node_id': '82e969e898a4593bfe1a4fdfecff14ef0ec554f194ae696ef63ce318'}

In [8]:
from ray.tune.registry import register_env
from rlskyjo.environment import skyjo_env
from ray.rllib.env import PettingZooEnv
import ray.rllib.agents.dqn as dqn
from ray.rllib.agents import ppo
from copy import deepcopy
from ray.rllib.agents.registry import get_agent_class
from ray.rllib.models import ModelCatalog
from rlskyjo.models.action_mask_model import TorchActionMaskModel
from ray.rllib.models import ModelCatalog
from ray.rllib.agents.dqn import DQNTrainer, DQNTorchPolicy
from ray.tune.logger import pretty_print
import os
import torch


In [9]:

env_name  = "pettingzoo_skyjo"
def env_creator():
        env = skyjo_env.env(**skyjo_env_cfg)
        return env

register_env(env_name,
                lambda config: PettingZooEnv(env_creator()))
ModelCatalog.register_custom_model(
        "pa_model2", TorchActionMaskModel
    )
env = PettingZooEnv(env_creator())

/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:52: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn("The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.")


In [10]:
env.env.observe(env.env.agent_selection)

{'observations': array([ 4, 10,  1,  0,  0,  1,  0,  1,  0,  0,  1,  0,  0,  1,  0,  1,  1,
        11, 15, 15, 15, 15, 15, 15,  6, 15, -2, 15, 15, 15, 15, 15, 15, 15,
         9,  3, 15, 15, 15, 15, 15, 15, 15, 15, 12, 15, 15,  1, 15, 15, 15,
        15, 15, 15, 15], dtype=int8),
 'action_mask': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1], dtype=int8)}

with multiagent

In [11]:

custom_config={
    "env":env_name,
    "model": {
        "custom_model": "pa_model2",
    },
    "framework": "torch",
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    "num_gpus": int(torch.cuda.device_count()),
    "num_workers": os.cpu_count() - 1,
    "multiagent":{
            "policies": {
                name: (None, env.observation_space, env.action_space, {}) for name in env.agents
            },
            "policy_mapping_fn": lambda agent_id: agent_id
    }
}


ppo_config = ppo.DEFAULT_CONFIG.copy()
ppo_config.update(custom_config)

trainer = ppo.PPOTrainer(config=ppo_config)

2022-01-30 10:49:51,590	WARNING ppo.py:143 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=15 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 266.
2022-01-30 10:49:51,591	INFO ppo.py:166 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-01-30 10:49:51,591	INFO trainer.py:743 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=595) /home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
(RolloutWorker pid=595)   warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
(RolloutWorker pid=595) /home/mich

In [12]:
# run manual training loop and print results after each iteration
max_steps = 1e8
max_iters = 1000
for iters in range(max_iters):
    result = trainer.train()
    if iters % 2 ==0:
        print(pretty_print(result))
    # stop training if the target train steps or reward are reached
    if result["timesteps_total"] >= max_steps:
        print(f"training done, because max_steps {max_steps} {result['timesteps_total']} reached")
        break
else:
    print(f"training done, because max_iters {max_iters} reached")
# manual test loop
print("Finished training. Running manual test/inference loop.")
# prepare environment with max 10 steps


(RolloutWorker pid=598) 2022-01-30 10:50:17,655	WARNING deprecation.py:45 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in the future!
(RolloutWorker pid=598) /home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: overflow encountered in reduce
(RolloutWorker pid=598)   ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
(RolloutWorker pid=598) /home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: overflow encountered in reduce
(RolloutWorker pid=598)   ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


agent_timesteps_total: 7950
custom_metrics: {}
date: 2022-01-30_10-50-35
done: false
episode_len_mean: 102.12676056338029
episode_media: {}
episode_reward_max: 3.000000000000014
episode_reward_mean: 3.0000000000000018
episode_reward_min: 2.999999999999986
episodes_this_iter: 71
episodes_total: 71
experiment_id: 3e2b5b7f5b6d4c4695fedd0410301df6
hostname: lenovo-michi
info:
  learner:
    player_0:
      learner_stats:
        cur_kl_coeff: 0.2
        cur_lr: 5.000000000000001e-05
        entropy: 1.7233023464679718
        entropy_coeff: 0.0
        kl: 0.010768052894116928
        policy_loss: -0.0023573344200849534
        total_loss: 673.5334084828695
        vf_explained_var: 0.002090949813524882
        vf_loss: 673.5336132303873
    player_1:
      learner_stats:
        cur_kl_coeff: 0.2
        cur_lr: 5.000000000000001e-05
        entropy: 1.733982622027397
        entropy_coeff: 0.0
        kl: 0.009977537440420037
        policy_loss: -0.027513244980946185
        total_loss

E0130 11:20:23.525907997   16901 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1643538023.525883450","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1643538023.525880635","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}


agent_timesteps_total: 885750
custom_metrics: {}
date: 2022-01-30_11-20-25
done: false
episode_len_mean: 476.22
episode_media: {}
episode_reward_max: 3.000000000000014
episode_reward_mean: 2.9999999999999987
episode_reward_min: 2.999999999999986
episodes_this_iter: 21
episodes_total: 3957
experiment_id: 3e2b5b7f5b6d4c4695fedd0410301df6
hostname: lenovo-michi
info:
  learner:
    player_0:
      learner_stats:
        cur_kl_coeff: 0.45
        cur_lr: 5.000000000000001e-05
        entropy: 0.6925105930368105
        entropy_coeff: 0.0
        kl: 0.017548427924666612
        policy_loss: -0.08546598446244995
        total_loss: 34.05137357791265
        vf_explained_var: 0.1613698453704516
        vf_loss: 34.12894288857778
    player_1:
      learner_stats:
        cur_kl_coeff: 0.45
        cur_lr: 5.000000000000001e-05
        entropy: 0.670968784938256
        entropy_coeff: 0.0
        kl: 0.018570603022213465
        policy_loss: -0.07049478913152901
        total_loss: 30.572023

E0130 11:32:18.527514652   23208 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1643538738.527488132","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1643538738.527483974","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}


...

agent_timesteps_total: 5961030
custom_metrics: {}
date: 2022-01-30_14-09-49
done: false
episode_len_mean: 122.76
episode_media: {}
episode_reward_max: 3.000000000000007
episode_reward_mean: 2.9999999999999996
episode_reward_min: 2.999999999999993
episodes_this_iter: 69
episodes_total: 34759
experiment_id: 3e2b5b7f5b6d4c4695fedd0410301df6
hostname: lenovo-michi
info:
  learner:
    player_0:
      learner_stats:
        cur_kl_coeff: 0.675
        cur_lr: 5.000000000000001e-05
        entropy: 0.4835018185774485
        entropy_coeff: 0.0
        kl: 0.017716326884892622
        policy_loss: -0.07225073617883027
        total_loss: 46.894975295066835
        vf_explained_var: 0.4210588486989339
        vf_loss: 46.955267523129784
    player_1:
      learner_stats:
        cur_kl_coeff: 0.675
        cur_lr: 5.000000000000001e-05
        entropy: 0.4854671495159467
        entropy_coeff: 0.0
        kl: 0.015982580984812672
        policy_loss: -0.07472864480068286
        total_loss: 47

E0130 14:18:23.850987829   14905 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1643548703.850963502","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1643548703.850960937","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}


agent_timesteps_total: 6232351
custom_metrics: {}
date: 2022-01-30_14-18-47
done: false
episode_len_mean: 114.0
episode_media: {}
episode_reward_max: 3.000000000000014
episode_reward_mean: 3.0
episode_reward_min: 2.999999999999993
episodes_this_iter: 69
episodes_total: 37163
experiment_id: 3e2b5b7f5b6d4c4695fedd0410301df6
hostname: lenovo-michi
info:
  learner:
    player_0:
      learner_stats:
        cur_kl_coeff: 0.675
        cur_lr: 5.000000000000001e-05
        entropy: 0.4642292709151904
        entropy_coeff: 0.0
        kl: 0.015913048648345314
        policy_loss: -0.06275642617295185
        total_loss: 100.35060678005219
        vf_explained_var: 0.22782085021336873
        vf_loss: 100.40262151877086
    player_1:
      learner_stats:
        cur_kl_coeff: 0.675
        cur_lr: 5.000000000000001e-05
        entropy: 0.4978249167899291
        entropy_coeff: 0.0
        kl: 0.014900731008480458
        policy_loss: -0.09161695594433696
        total_loss: 101.7556837034225

E0130 15:12:49.009127654   11551 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1643551969.009091716","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1643551969.009085233","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}


agent_timesteps_total: 7876230
custom_metrics: {}
date: 2022-01-30_15-13-20
done: false
episode_len_mean: 104.98
episode_media: {}
episode_reward_max: 3.000000000000007
episode_reward_mean: 2.9999999999999987
episode_reward_min: 2.999999999999986
episodes_this_iter: 79
episodes_total: 51913
experiment_id: 3e2b5b7f5b6d4c4695fedd0410301df6
hostname: lenovo-michi
info:
  learner:
    player_0:
      learner_stats:
        cur_kl_coeff: 0.675
        cur_lr: 5.000000000000001e-05
        entropy: 0.4677635258436203
        entropy_coeff: 0.0
        kl: 0.0165134158967993
        policy_loss: -0.10777415073476732
        total_loss: 57.29183735370636
        vf_explained_var: 0.44846080015103024
        vf_loss: 57.38846488952637
    player_1:
      learner_stats:
        cur_kl_coeff: 0.675
        cur_lr: 5.000000000000001e-05
        entropy: 0.47894075165192285
        entropy_coeff: 0.0
        kl: 0.01765352508732273
        policy_loss: -0.07581834174071748
        total_loss: 62.46

In [69]:
from rlskyjo.game.skyjo import SkyjoGame

env = PettingZooEnv(env_creator())
obs = env.reset()
done = {'__all__': False}
# run one iteration until done

def rec_deep(env, attiribute = "table"):
    if hasattr(env, attiribute):
        return getattr(env, attiribute)
    else:
        return rec_deep(env.env, attiribute)

stop = False
for i in range(2000):
    for i in range(1000):
        if done["__all__"]:
            # print("game done")
            break
        # get agent from current observation
        agent = list(obs.keys())[0]

        # format observation dict
        # print(obs)
        obs = obs[agent]
        # env.render()
        
        # get deterministic action
        # trainer.compute_single_action(obs, policy_id=agent)
        policy = trainer.get_policy(policy_id=agent)
        action_exploration_policy, _, action_info = policy.compute_single_action(obs)
        # logits = action_info['action_dist_inputs']
        # action = logits.argmax()
        # 
        action = action_exploration_policy
        # print("agent ", agent, " action ", SkyjoGame.render_action_explainer(action))
        obs, reward, done, _ = env.step({agent: action})
        # observations contain original observations and the action mask
        # print(f"Obs: {obs}, Action: {action}, done: {done}")

    table = rec_deep(env, "table")

    if any(table.game_metrics["num_refunded"]):
        env.render()
        print(table.game_metrics)
        break
    
    

/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:44: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn("The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.")
/home/michi/anaconda3/envs/skybo/lib/python3.9/site-packages/pettingzoo/utils/wrappers/base.py:52: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn("The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.")


In [54]:
table.game_metrics["num_refunded"]
table._evaluate_game(table.players_cards, 0, score_penalty=2), table.players_cards


([76.0, 41.0, 21.0],
 array([[-1,  9,  7, -2,  4,  2,  0,  7,  4,  0,  3,  5],
        [ 0,  7,  1, 10,  7,  2,  0,  6,  1, -1, -1,  9],
        [-1,  6,  5, -2,  4,  2,  1,  4, -2,  3, -2,  3]], dtype=int8))

In [42]:
agent = "player_0"
obs = { agent: {'observations': np.array([27,  0,  3,  
        3,  6,  6,  3,  6,  4,  4,  6,  2,  2,  6,  4,  5,  2, 4, 
        12, # holding card
        # player 0
        1, 15, 15, 
        -1, 4, -2, 
        12, 15, 12,
        1, 0, 0,
        # player 1
         9, 0, 2,
         1, -1, 3,
         1,-2, 5,
         -2,15,11, 
        # player 2
        -1, 9, 6,
        2, 3, 0,
        1, 0, 1,
        7, 7,15
        ], dtype=np.int8), 
 'action_mask': np.array(
     [ # put
      1, 1, 1, 
      1, 1, 1, 
      1, 1, 1, 
      1, 1, 1,
      # reveal
      0, 1, 1, 
      0, 0, 0, 
      0, 1, 0, 
      0, 0, 0,
      # draw
      1, 1], dtype=np.int8)}}

action = trainer.compute_single_action(obs[agent], policy_id=agent)
print("agent ", agent, " action ", SkyjoGame.render_action_explainer(action))
action, agent

agent  player_0  action  place card (1) - col:0 row:1


(1, 'player_0')

Save trainer

In [71]:
import ray.tune
import os 
from rlskyjo.utils import get_project_root
analysis = ray.tune.run(ppo.PPOTrainer, config=ppo_config, local_dir=os.path.join(get_project_root(), "models"), stop={"training_iteration": 10},
                            checkpoint_at_end=True)

Trial name,status,loc
PPO_pettingzoo_skyjo_8e6bd_00000,PENDING,


Trial name,status,loc
PPO_pettingzoo_skyjo_8e6bd_00000,PENDING,


(scheduler +6h10m10s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +6h10m10s) Error: No available node types can fulfill resource request {'GPU': 1.0, 'CPU': 1.0}. Add suitable node types to this cluster to resolve this issue.


Trial name,status,loc
PPO_pettingzoo_skyjo_8e6bd_00000,PENDING,


Trial name,status,loc
PPO_pettingzoo_skyjo_8e6bd_00000,PENDING,


...

Trial name,status,loc
PPO_pettingzoo_skyjo_8e6bd_00000,PENDING,


(scheduler +6h21m51s) Error: No available node types can fulfill resource request {'GPU': 1.0, 'CPU': 1.0}. Add suitable node types to this cluster to resolve this issue.


2022-01-30 17:10:54,025	WARNING tune.py:582 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Trial name,status,loc
PPO_pettingzoo_skyjo_8e6bd_00000,PENDING,


2022-01-30 17:10:54,248	ERROR tune.py:622 -- Trials did not complete: [PPO_pettingzoo_skyjo_8e6bd_00000]
2022-01-30 17:10:54,249	INFO tune.py:626 -- Total run time: 706.40 seconds (706.16 seconds for the tuning loop).
2022-01-30 17:10:54,249	WARNING tune.py:630 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
E0130 17:55:28.589606166    5678 backup_poller.cc:134]       Run client channel backup poller: {"created":"@1643561728.588626256","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1643561728.588618531","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}
